# Análisis exploratorio de datos 2

En este notebook analizamos los datos obtenidos por AED_2, que son datos de tiempo y no de voltaje de cada onda. <br>
El plan es no eliminar outliers en esta fase del proyecto, pero si ajustar los límites de los gráficos para que se vean más claros. <br>
Los datos de voltajes tienen 3 id_ecg más que los datos de tiempos (son los id_ecg = 1802, 2171, 6897). Estos ecg tienen hoyos en su información, por eso no puede proceder correctamente las herramientas de neurokit2.

In [ ]:
# paquetes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

Para ejecutar este código, es necesario primero ejecutar el markdown "Obtencion_Datos.ipynb".

In [ ]:
# importación de los datos
datos = pd.read_csv('df_datos_tiempos.csv', index_col='Unnamed: 0')

# un resumen rápido de la información del df
datos.info()

## Boxplots

Los boxplots de seaborn muestran la siguiente información
- mediana (línea de en medio).
- Q1 y Q3 (límites de la caja).
- Q1 - 1.5*IQR (límite del bigote inferior...).
- Q3 + 1.5*IQR (... y superior, donde IQR = InterQuartile Ratio = Q3-Q1).
- Outliers (datos fuera de los bigotes).

In [ ]:
# creamos la figura
fig, axs = plt.subplots(6,4, figsize=(15, 22))

# agregamos los gráficos uno por uno
z = 0 # para obtener solo una vez la info de leyendas
# flatten nos va a ayudar a ordenar unidimensionalmente la ubicación de los axis
# zip nos va a ayudar a regresar en pares ordenados la info de col,ubi
for columna, ubicacion in zip(datos.columns[2:26],axs.flatten()):
    #sacamos Q1,Q2 y el rango intercuartilico para definir los límites del eje x
    Q1,Q3 = np.quantile(datos[columna].dropna(),[0.25,0.75])
    RIQ = Q3-Q1
    li = Q1 - 3*RIQ
    ls = Q3 + 4*RIQ
    sns.boxplot(datos, x=columna, hue='categoria', ax=ubicacion)
    ubicacion.set_xlim([li,ls])
    ubicacion.get_legend().remove() # quitamos las leyendas individuales de los gráficos
    if z == 0: # para el primer ciclo...
        handles, labels = ubicacion.get_legend_handles_labels() # obtenemos la info de leyendas
        z =+ 1 # cambiamos z para no volver a sacar la info
# agregamos la leyenda global
fig.legend(handles, labels, loc='upper center', ncols = 4, bbox_to_anchor=(0.5, 0.92))
plt.show() # imprimimos la imagen

## Correlación de las variables contínuas

In [ ]:
# matríz de correlación
fig, ax = plt.subplots(figsize=(10,10))
datos_correlacion = datos[datos.columns[2:26]].corr()
sns.heatmap(datos_correlacion,
            annot=True,
            fmt=".1f",
            cmap=sns.color_palette("vlag",as_cmap=True),
            xticklabels=datos_correlacion.columns.values,
            yticklabels=datos_correlacion.columns.values)
plt.show()

In [ ]:
# representación gráfica de las correlaciones
sns.pairplot(data=datos, hue='categoria', vars=datos.columns[2:26])
plt.show()

## Análisis de Componentes Principales

In [ ]:
# PCA

# primero arreglamos los datos
datos_pca_originales = datos.drop(['id_ecg','categoria','patient_id','sex'],axis=1).dropna() # quitamos algunas columnas no numéricas y los NA.
scaler = StandardScaler() # llamamos al normalizador
datos_pca = scaler.fit_transform(datos_pca_originales) # normalizamos

# aplicamos PCA
pca = PCA(n_components=2) # llamamos al PCAdor
datos_pca = pca.fit_transform(datos_pca) # aplicamos PCA

# calculamos los pesos de cada variable en cada componente (se usan abajo)
componentes = pca.components_  # Matriz de pesos: cada fila es un componente (PC1, PC2, ...)
componentes = pd.DataFrame(componentes, columns=datos_pca_originales.columns) # transformamos los valores en un df
print(f"\nCargas de las variables en los componentes principales:\n{componentes}")

In [ ]:
# scatterplot del PCA
plt.figure(figsize=(8, 6))        # tamaño del gráfico
plt.scatter(x = datos_pca[:, 0],  # eje x será el PCA1
            y = datos_pca[:, 1],  # eje y será el PCA2
            c='blue',                 # color de los puntitos
            edgecolor='k',            # color del perímetro de los puntitos. k es negro
            s=60)                     # tamaño de los puntitos             
plt.title('Proyección de los Datos en el Espacio de los Componentes Principales')
plt.xlabel('Primer Componente Principal (PC1)')
plt.ylabel('Segundo Componente Principal (PC2)')
plt.grid()
plt.show()

In [ ]:
# gráfico de barras con el peso de cada variable en cada componente principal
plt.figure(figsize=(8, 6))
componentes.T.plot(kind='bar', figsize=(10, 6), legend=True)
plt.title('Peso de cada variable en los Componentes Principales (PC1, PC2)')
plt.ylabel('Cargas (Pesos)')
plt.xlabel('Variables')
plt.grid(axis='y')
plt.xticks(rotation=45)
plt.show()

**Nota:** No se incluyen datos de edad y sexo al ser los mismos que en AED.